## This notebook shows how to use Orchestrator APIs for user experiments

In [26]:
import os
from fabric_cm.credmgr.credmgr_proxy import CredmgrProxy
from fabric_cf.orchestrator.orchestrator_proxy import OrchestratorProxy
import json

In [27]:
credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
credmgr_proxy = CredmgrProxy(credmgr_host=credmgr_host)
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
orchestrator_proxy = OrchestratorProxy(orchestrator_host=orchestrator_host)

### Fabric Tokens
Fabric has 2 kinds of tokens:<br>
- Identity : required for Control/Measurement Framework APIs. Identity Token is valid upto an hour.
- Refresh : required to generate new Identity Tokens valid. Refresh Token is valid for 24 hours.

Fabric Identity token is required for Control/Measurement Framework APIs.<br>

When user logins to Jupyterhub after authenticating against CILogon, OIDC refresh token is derived. <br>
This token is available as the environment variable `CILOGON_REFRESH_TOKEN`.<br>

On the first login, we use `CILOGON_REFRESH_TOKEN` to generate new Fabric Identity Token and Fabric Refresh Token.<br>
For any subsequent use, we use Fabric Refresh Token. On every refresh, Fabric Refresh Token is changed and updated.<br>

NOTE: These steps are required for any experiments on Fabric Testbed.<br>

In [28]:
not_found=False
fabric_refresh_token=None
%store -r fabric_refresh_token

if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token
print("Fabric Refresh Token {}".format(fabric_refresh_token))
print("CILOGON_REFRESH_TOKEN environment variable: {}".format(os.environ['CILOGON_REFRESH_TOKEN']))

Fabric Refresh Token https://cilogon.org/oauth2/refreshToken/2b42beb4c017c99deedc6a1cd2faa848/1615398068367
CILOGON_REFRESH_TOKEN environment variable: https://cilogon.org/oauth2/refreshToken/7de2a29bceb35047a6b92e5273ee7d88/1615398021798


### Get new Fabric Identity Token and update Fabric Refresh Token

In [29]:
try:
    refresh_res = credmgr_proxy.refresh(project_name='all', scope='all', refresh_token=fabric_refresh_token)
    print("New Tokens received: {}".format(json.dumps(refresh_res)))
    fabric_id_token=refresh_res['id_token']
    fabric_refresh_token=refresh_res['refresh_token']
    print()
    print("New Refresh Token: {}".format(fabric_refresh_token))
    print()
    print("Stored new Refresh Token")
    %store fabric_refresh_token
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

New Tokens received: {"id_token": "eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsImtpZCI6ImI0MTUxNjcyMTExOTFlMmUwNWIyMmI1NGIxZDNiNzY2N2U3NjRhNzQ3NzIyMTg1ZTcyMmU1MmUxNDZmZTQzYWEifQ.eyJlbWFpbCI6Imt0aGFyZTEwQGVtYWlsLnVuYy5lZHUiLCJjZXJ0X3N1YmplY3RfZG4iOiIvREM9b3JnL0RDPWNpbG9nb24vQz1VUy9PPVVuaXZlcnNpdHkgb2YgTm9ydGggQ2Fyb2xpbmEgYXQgQ2hhcGVsIEhpbGwvQ049S29tYWwgVGhhcmVqYSBBMTE5MDQxMDYiLCJpZHAiOiJ1cm46bWFjZTppbmNvbW1vbjp1bmMuZWR1IiwiaWRwX25hbWUiOiJVbml2ZXJzaXR5IG9mIE5vcnRoIENhcm9saW5hIGF0IENoYXBlbCBIaWxsIiwiZXBwbiI6Imt0aGFyZTEwQHVuYy5lZHUiLCJhZmZpbGlhdGlvbiI6ImVtcGxveWVlQHVuYy5lZHU7bWVtYmVyQHVuYy5lZHU7c3RhZmZAdW5jLmVkdSIsIm5hbWUiOiJLb21hbCBUaGFyZWphIiwiYWNyIjoidXJuOm9hc2lzOm5hbWVzOnRjOlNBTUw6Mi4wOmFjOmNsYXNzZXM6UGFzc3dvcmRQcm90ZWN0ZWRUcmFuc3BvcnQiLCJlbnRpdGxlbWVudCI6InVybjptYWNlOmRpcjplbnRpdGxlbWVudDpjb21tb24tbGliLXRlcm1zIiwiaXNzIjoiaHR0cHM6Ly9jaWxvZ29uLm9yZyIsInN1YiI6Imh0dHA6Ly9jaWxvZ29uLm9yZy9zZXJ2ZXJBL3VzZXJzLzExOTA0MTAxIiwiYXVkIjoiY2lsb2dvbjovY2xpZW50X2lkLzczYmI1MDM4MjlmZjk2YWQ5NDhmNDFiMWQ4ZTJhMmE1Ii

### Orchestrator API example to query for available resources

In [30]:
resources = orchestrator_proxy.resources(token=fabric_id_token)

print(resources)

(<Status.OK: 1>, {'value': {'bqm': '<?xml version="1.0" encoding="UTF-8"?><graphml '
                  'xmlns="http://graphml.graphdrawing.org/xmlns" '
                  'xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" '
                  'xsi:schemaLocation="http://graphml.graphdrawing.org/xmlns '
                  'http://graphml.graphdrawing.org/xmlns/1.0/graphml.xsd"><key '
                  'id="GraphID" for="node" attr.name="GraphID" '
                  'attr.type="string"/><key id="labels" for="node" '
                  'attr.name="labels" attr.type="string"/>\n'
                  '<key id="Labels" for="node" attr.name="Labels" '
                  'attr.type="string"/><key id="label" for="node" '
                  'attr.name="label" attr.type="string"/><key id="Capacities" '
                  'for="node" attr.name="Capacities" attr.type="string"/><key '
                  'id="Layer" for="node" attr.name="Layer" '
                  'attr.type="string"/><key id="labels" for=

## Create Slice

In [31]:
import fim.user as fu
# Create topology
t = fu.ExperimentTopology()

# Add node
n1 = t.add_node(name='n1', site='RENC')

# Set capacities
cap = fu.Capacities()
cap.set_fields(core=4, ram=64, disk=500)

# Set Properties
n1.set_properties(capacities=cap, image_type='qcow2', image_ref='default_centos_8')

# Add PCI devices
n1.add_component(ctype=fu.ComponentType.SmartNIC, model='ConnectX-6', name='nic1')

# Add node
n2 = t.add_node(name='n2', site='RENC')

# Set properties
n2.set_properties(capacities=cap, image_type='qcow2', image_ref='default_centos_8')

# Add PCI devices
n2.add_component(ctype=fu.ComponentType.GPU, model='RTX6000', name='nic2')

# Generate Slice Graph
slice_graph = t.serialize()

# Request slice from Orchestrator
status, result = orchestrator_proxy.create(token=fabric_id_token, slice_name='JupyterSlice1', slice_graph=slice_graph)

print("Response Status {}".format(status))
print("Response received {}".format(result))

Response Status Status.OK
Response received {'value': {'message': '',
           'reservations': [{'pending_state': 'None_',
                             'reservation_id': '19e67e89-459a-45f9-b54e-cd4ac3c0e136',
                             'resource_type': 'VM',
                             'slice_id': 'dd072880-6ccf-45fa-82d2-8e7746648a82',
                             'state': 'Unknown'},
                            {'pending_state': 'None_',
                             'reservation_id': 'ee461498-23ae-4565-a063-1479104150c3',
                             'resource_type': 'VM',
                             'slice_id': 'dd072880-6ccf-45fa-82d2-8e7746648a82',
                             'state': 'Unknown'}],
           'status': 'success'}}


In [32]:
# Set the Slice ID from output of the above command
slice_id='dd072880-6ccf-45fa-82d2-8e7746648a82'

## Query Slices

In [33]:
status, result = orchestrator_proxy.slices(token=fabric_id_token)

print("Response Status {}".format(status))
print("Response received {}".format(result))

Response Status Status.OK
Response received {'value': {'message': '',
           'slices': [{'graph_id': 'c2e22076-c4b2-47a9-ac70-1a1c57706887',
                       'slice_id': 'dd072880-6ccf-45fa-82d2-8e7746648a82',
                       'slice_name': 'JupyterSlice1',
                       'slice_state': 'StableOK'}],
           'status': 'success'}}


## Query Slice

In [34]:
status, result = orchestrator_proxy.get_slice(token=fabric_id_token, slice_id=slice_id)

print("Response Status {}".format(status))
print("Response received {}".format(result))

Response Status Status.OK
Response received {'value': {'message': '',
           'slice_model': '<?xml version="1.0" encoding="UTF-8"?><graphml '
                          'xmlns="http://graphml.graphdrawing.org/xmlns" '
                          'xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" '
                          'xsi:schemaLocation="http://graphml.graphdrawing.org/xmlns '
                          'http://graphml.graphdrawing.org/xmlns/1.0/graphml.xsd"><key '
                          'id="Site" for="node" attr.name="Site" '
                          'attr.type="string"/><key id="labels" for="node" '
                          'attr.name="labels" attr.type="string"/>\n'
                          '<key id="instance_name" for="node" '
                          'attr.name="instance_name" attr.type="string"/><key '
                          'id="bqm_node_id" for="node" attr.name="bqm_node_id" '
                          'attr.type="string"/><key id="ImageRef" for="node" '
  

## Query Slivers

In [37]:
status, result = orchestrator_proxy.slivers(token=fabric_id_token, slice_id=slice_id)

print("Response Status {}".format(status))
print("Response received {}".format(result))

Response Status Status.OK
Response received {'value': {'message': '',
           'reservations': [{'join_state': 'NoJoin',
                             'pending_state': 'None_',
                             'reservation_id': '19e67e89-459a-45f9-b54e-cd4ac3c0e136',
                             'resource_type': 'VM',
                             'slice_id': 'dd072880-6ccf-45fa-82d2-8e7746648a82',
                             'state': 'Active'},
                            {'join_state': 'NoJoin',
                             'pending_state': 'None_',
                             'reservation_id': 'ee461498-23ae-4565-a063-1479104150c3',
                             'resource_type': 'VM',
                             'slice_id': 'dd072880-6ccf-45fa-82d2-8e7746648a82',
                             'state': 'Active'}],
           'status': 'success'}}


In [38]:
# Set the Sliver ID from output of the above command by capturing reservation_id

sliver_id='19e67e89-459a-45f9-b54e-cd4ac3c0e136'

## Query Sliver

In [39]:
status, result = orchestrator_proxy.slivers(token=fabric_id_token, slice_id=slice_id, sliver_id=sliver_id)

print("Response Status {}".format(status))
print("Response received {}".format(result))

Response Status Status.OK
Response received {'value': {'message': '',
           'reservations': [{'join_state': 'NoJoin',
                             'pending_state': 'None_',
                             'reservation_id': '19e67e89-459a-45f9-b54e-cd4ac3c0e136',
                             'resource_type': 'VM',
                             'slice_id': 'dd072880-6ccf-45fa-82d2-8e7746648a82',
                             'state': 'Active'}],
           'status': 'success'}}


## Slice Status

In [40]:
status, result = orchestrator_proxy.slice_status(token=fabric_id_token, slice_id=slice_id)

print("Response Status {}".format(status))
print("Response received {}".format(result))

Response Status Status.OK
Response received {'value': {'message': '',
           'slices': [{'graph_id': 'c2e22076-c4b2-47a9-ac70-1a1c57706887',
                       'slice_id': 'dd072880-6ccf-45fa-82d2-8e7746648a82',
                       'slice_name': 'JupyterSlice1',
                       'slice_state': 'StableOK'}],
           'status': 'success'}}


## Sliver Status

In [43]:
status, result = orchestrator_proxy.sliver_status(token=fabric_id_token, slice_id=slice_id, sliver_id=sliver_id)

print("Response Status {}".format(status))
print("Response received {}".format(result))

Response Status Status.OK
Response received {'value': {'message': '',
           'reservations': [{'join_state': 'NoJoin',
                             'notices': 'Reservation '
                                        '19e67e89-459a-45f9-b54e-cd4ac3c0e136 '
                                        '(Slice '
                                        'JupyterSlice1(dd072880-6ccf-45fa-82d2-8e7746648a82) '
                                        'Graph '
                                        'Id:c2e22076-c4b2-47a9-ac70-1a1c57706887 '
                                        ') is in state [Active,None_]\n'
                                        '\n'
                                        'Last ticket update: \n'
                                        'Ticket events: \n'
                                        'Last ticket update: \n'
                                        'Ticket events: ',
                             'pending_state': 'None_',
                             'reservation_i

## Delete Slice

In [45]:
status, result = orchestrator_proxy.delete(token=fabric_id_token, slice_id=slice_id)

print("Response Status {}".format(status))
print("Response received {}".format(result))

Response Status Status.OK
Response received {'value': None}
